In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-06"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13769,2024-03-06,Eua Nba,21:30,Atlanta Hawks,Cleveland Cavaliers,2.10,1.79,220.5,1.80,2.01,1.5,1.97,1.74,nJoN6wWp,0.476190,0.558659,0.555556,0.497512,0.034850,0.4,0.2,NaN,NaN,192.242,35.520646,0.184770,160.976,26.797079,0.166466,225.206,154.532,205.84,193.11,0.0,0.0,0.0,0.0,0.112701,0.077949,0.087674,0.02,0.004,275.000000,0.558726,0.6,0.041274,-1.29,-0.258,-3.062016,0.585217,0.6,0.014783
13770,2024-03-06,Eua Nba,23:00,Utah Jazz,Chicago Bulls,1.71,1.98,228.5,1.97,1.97,-2.5,2.05,2.14,dlTDT0wb,0.584795,0.505051,0.507614,0.507614,0.089846,0.4,0.8,NaN,NaN,253.762,65.197055,0.256922,356.508,83.239569,0.233486,227.564,321.228,269.89,464.10,0.0,0.0,0.0,0.0,0.103479,0.000000,0.030377,-1.09,-0.218,-3.256881,0.551319,0.6,0.048681,-1.25,-0.250,-3.920000,0.431377,0.4,-0.031377
13771,2024-03-06,Austrália Liga Feminina Wnbl,05:00,Southside F,Melbourne Boomers F,1.66,2.17,164.5,1.87,1.85,-3.5,1.91,2.45,Es95XcAN,0.602410,0.460829,0.534759,0.540541,0.063239,0.4,0.4,NaN,NaN,130.798,24.339320,0.186083,156.430,72.090059,0.460845,114.436,113.822,116.48,83.57,0.0,0.0,0.0,0.0,0.188316,0.007603,0.175155,-0.96,-0.192,-3.437500,0.000000,0.0,0.000000,-1.36,-0.272,-4.301471,0.000000,0.0,0.000000
13772,2024-03-06,Áustria Superliga,15:00,St. Polten,Wels,2.14,1.68,155.5,1.87,1.79,1.5,1.97,1.52,4MKgemsi,0.467290,0.595238,0.534759,0.558659,0.062528,0.8,0.2,NaN,NaN,182.308,58.434371,0.320526,140.144,44.527575,0.317727,128.976,158.722,142.20,123.76,0.0,0.0,0.0,0.0,0.170298,0.030912,0.182348,-3.98,-0.796,-1.432161,0.000000,0.0,0.000000,-0.88,-0.176,-3.863636,0.000000,0.0,0.000000
13773,2024-03-06,Eslovênia 1. Skl Feminino,13:00,Jezica F,Maribor F,1.97,1.76,142.5,1.87,1.79,1.5,1.83,1.62,SvxeFX5l,0.507614,0.568182,0.534759,0.558659,0.075796,0.8,1.0,NaN,NaN,218.556,292.960080,1.340435,203.592,185.385395,0.910573,251.626,193.050,741.60,124.32,1.0,0.0,0.0,0.0,0.079621,0.030912,0.086083,-1.13,-0.226,-4.292035,0.000000,0.0,0.000000,-0.08,-0.016,-47.500000,0.000000,0.0,0.000000
13774,2024-03-06,Eua Ncaa,01:00,Boise State,Nevada,1.41,2.87,139.5,1.83,1.83,-6.5,2.00,2.00,0zag5W52,0.709220,0.348432,0.546448,0.546448,0.057652,0.6,0.4,NaN,NaN,214.010,177.145745,0.827745,158.724,67.746701,0.426821,112.796,194.128,97.20,274.89,0.0,0.0,0.0,0.0,0.482419,0.000000,0.000000,-0.79,-0.158,-2.594937,0.776225,0.8,0.023775,4.54,0.908,2.059471,0.522110,0.7,0.177890
13775,2024-03-06,Eua Ncaa,01:00,UNLV,San Diego State,2.18,1.69,135.5,1.83,1.83,1.5,1.91,1.63,v7gQBpO7,0.458716,0.591716,0.546448,0.546448,0.050432,0.6,0.6,NaN,NaN,177.312,175.229614,0.988256,113.006,16.543604,0.146396,108.520,127.294,118.80,129.78,0.0,0.0,0.0,0.0,0.179061,0.000000,0.111859,-2.47,-0.494,-2.388664,0.595089,0.6,0.004911,-3.81,-0.762,-0.905512,0.000000,0.0,0.000000
13776,2024-03-06,Eua Ncaa,21:00,Arkansas,LSU,1.73,2.10,155.5,1.83,1.83,-2.5,1.83,2.40,K2WCZmGE,0.578035,0.476190,0.546448,0.546448,0.054225,0.4,0.6,NaN,NaN,274.714,121.103229,0.440834,219.412,113.300959,0.516385,153.994,400.668,250.11,191.36,0.0,0.0,0.0,0.0,0.136621,0.000000,0.190568,-3.31,-0.662,-1.102719,0.607796,0.6,-0.007796,-2.01,-0.402,-2.736318,0.000000,0.0,0.000000
13777,2024-03-06,Eua Ncaa,21:00,Davidson,Loyola (Ill).,1.90,1.90,134.5,1.83,1.83,-1.5,2.00,2.03,0CrPQqbE,0.526316,0.526316,0.546448,0.546448,0.052632,0.0,0.8,NaN,NaN,112.348,30.577689,0.272169,122.662,27.666646,0.225552,103.012,164.418,91.12,132.84,0.0,0.0,0.0,0.0,0.000000,0.000000,0.010528,-2.39,-0.478,-1.882845,0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,23:00,Eua Nba,Utah Jazz,Chicago Bulls,228.5,1.97,1.0000,Over
1,05:00,Austrália Liga Feminina Wnbl,Southside F,Melbourne Boomers F,164.5,1.87,1.0000,Over
2,01:00,Eua Ncaa,Boise State,Nevada,139.5,1.83,1.0000,Over
3,21:00,Eua Ncaa,Arkansas,LSU,155.5,1.83,1.0000,Over
4,21:00,Eua Ncaa,Davidson,Loyola (Ill).,134.5,1.83,1.0000,Over
5,23:00,Eua Ncaa,Texas A&M,Mississippi St.,139.5,1.80,1.0000,Over
6,14:30,Europa Liga Dos Campeões,Peristeri,Pinar Karsiyaka,163.5,1.87,1.0000,Over
7,14:00,Europa Euroliga Feminina,DVTK F,ESB Villeneuve F,137.5,1.89,1.0000,Over
8,15:00,Europa Euroliga Feminina,USK Prague F,Schio F,138.5,1.89,1.0000,Over
9,13:30,Finlândia Korisliiga,BC Nokia,Helsinki Seagulls,168.5,1.86,1.0000,Over
